In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy

In [1]:
def simple_read_arduino(serial_port='COM9', baud_rate=9600, timeout=2):
    import serial
    with serial.Serial(port=serial_port, baudrate=baud_rate, timeout=timeout) as arduino:
        arduino.read_until(b'Yours sincerely, Arduino') # no longer bother printing this!
        data = arduino.read_until(b'\r\n') 
        try:
            numeric_data = float(data.decode().strip())
        except ValueError:
            numeric_data = 0.0     
    return numeric_data

In [2]:
def improved_read_arduino(number_readings=1, data_type=numpy.float32, error_value=numpy.NaN,
                          serial_port='COM9', baud_rate=9600, timeout=2,
                          terminator='\r\n', startup_message='Yours sincerely, Arduino'): 
    
    if number_readings < 1: number_readings = 1 #need at least 1 reading
    numeric_data = numpy.zeros((number_readings,), dtype=data_type)
    import serial
    with serial.Serial(port=serial_port, baudrate=baud_rate, timeout=timeout) as arduino:
        arduino.read_until(startup_message.encode()) # string.encode() -> bytearray
        for i in range(number_readings):
            data = arduino.read_until(terminator.encode()) # string.encode() -> bytearray
            try:
                numeric_data[i] = data_type(data.decode().strip())
            except ValueError:
                numeric_data[i] = error_value 
    return numeric_data

In [3]:
def plot(data):
    mean = numpy.mean(data) 
    std_error = numpy.std(data)/numpy.sqrt(len(data))
    print('average value =', mean, '+/-', std_error, 'units')

    plt.figure(figsize=(10,2))
    plt.plot(data, marker='o', linestyle='none')
    plt.axhline(mean-std_error, color='r')
    plt.axhline(mean, color='k')
    plt.axhline(mean+std_error, color='r')
    plt.xlabel('Sample no.'); plt.ylabel('A0 voltage (V)')
    plt.show()

In [4]:
def freq_gen(freq=120, length=10.0):
    '''
    freq -> signal frequency in Hz, 
    length -> signal duration in seconds
    '''
    import numpy as np
    from simpleaudio import play_buffer
    
    sample_rate = 44100 # Hz - CD quality
    samples = int(length)*sample_rate
    time = np.linspace(0, int(length), samples, False)
    
    audio = np.zeros((samples, 2))
    audio[:,0] = 1 + np.sin(2*np.pi*freq*time) # 'left' channel
    audio[:,1] = 1 + np.cos(2*np.pi*freq*time) # 'right' channel
    audio *= 32767 / np.max(np.abs(audio)) # normalize to 16-bit range
    audio = audio.astype(np.int16) # convert to 16-bit datatype

    # start playback
    play = play_buffer(audio, 2, 2, sample_rate)
    print('Playing {} s audio signal at {} Hz.'.format(int(length), freq))
    # DON'T wait for playback to finish before exiting

In [6]:
def plot_timestamp(signals, times):
    '''
    Function to display sampled data with timestamps (seconds).
    '''
    diffs = 1000*numpy.diff(times) # element-wise difference, in ms
    
    plt.figure(figsize=(10,3))
    plt.subplot(211)
    plt.plot(times, signals, label='A0') 
    plt.legend() 
    plt.xlabel('time (seconds)'); plt.ylabel('ADC\nVoltage (V)')

    plt.subplot(223)
    plt.plot(diffs)
    plt.xlabel('sample number'); plt.ylabel('Time difference\n(ms)')

    plt.subplot(224)
    plt.plot(1000*times) # ms
    plt.xlabel('sample number'); plt.ylabel('Time (ms)')

    plt.tight_layout()
    plt.show()
    
    print('Sample rate = {} samples/second'.format(len(times)/(times[-1]-times[0])))
    print('Time diffs:')
    print('Min:', numpy.min(diffs), 'ms')
    print('Mean:', numpy.mean(diffs), 'ms')
    print('Max:', numpy.max(diffs), 'ms')
    print('SD:', numpy.std(diffs), 'ms')

In [7]:
def read_arduino_pc(number_readings=1, data_type=numpy.float32, error_value=numpy.NaN,
                    serial_port='COM9', baud_rate=115200, timeout=2,
                    terminator='\r\n', startup_message='Yours sincerely, Arduino'): 
    
    if number_readings < 1: number_readings = 1 
    numeric_data = numpy.zeros((number_readings,), dtype=data_type)
    # NEW: PC-generated timestamps, in seconds, which we store as floating-point
    timestamps_s = numpy.zeros((number_readings,), dtype=numpy.float32)
        
    import serial
    # NEW: use datetime module to generate best possible timestamps from PC
    from datetime import datetime as dt 
    
    with serial.Serial(port=serial_port, baudrate=baud_rate, timeout=timeout) as arduino:
        arduino.read_until(startup_message.encode())
        # NEW: start timing on PC
        start_time = dt.now()        
        
        for i in range(number_readings):
            data = arduino.read_until(terminator.encode()) 
            # NEW: time when data received (since start of program) 
            timestamps_s[i] = (dt.now() - start_time).total_seconds()
            
            try:
                numeric_data[i] = data_type(data.decode().strip())
            except ValueError:
                numeric_data[i] = error_value 
    return numeric_data, timestamps_s

In [8]:
def read_arduino_stamp(number_readings=1, data_type=numpy.float32, error_value=numpy.NaN,
                       serial_port='COM9', baud_rate=115200, timeout=2,
                       terminator='\r\n', startup_message='Yours sincerely, Arduino'): 
    
    if number_readings < 1: number_readings = 1 
    numeric_data = numpy.zeros((number_readings,), dtype=data_type)
    # NEW: Arduino timestamp, in microseconds - unsigned 32-bit integer = unsigned long
    timestamps = numpy.zeros((number_readings,), dtype=numpy.uint32)
        
    import serial
    with serial.Serial(port=serial_port, baudrate=baud_rate, timeout=timeout) as arduino:
        arduino.read_until(startup_message.encode())
        for i in range(number_readings):
            data = arduino.read_until(terminator.encode()) 
            values = data.decode().strip().split(',')
            try:
                numeric_data[i] = data_type(values[0])
                timestamps[i] = numpy.uint32(values[1]) # NEW Arduino timestamp        
            except (ValueError, IndexError): # NEW: IndexError if no comma in data
                numeric_data[i] = error_value 
                timestamps[i] = 0 # NEW: can't use NaN as it is a 'float' not an 'int'
    
    return numeric_data, timestamps*1e-6 # NEW: return timestamp values in seconds

In [9]:
def read_arduino_diff(number_readings=1, data_type=numpy.float32, error_value=numpy.NaN,
                      serial_port='COM10', baud_rate=115200, timeout=2,
                      terminator='\r\n', startup_message='Yours sincerely, Arduino'): 
    
    if number_readings < 1: number_readings = 1 
    numeric_data = numpy.zeros((number_readings,), dtype=data_type)
    # NEW: Arduino time differences, in microseconds - unsigned 16-bit integer = unsigned int
    time_diffs = numpy.zeros((number_readings,), dtype=numpy.uint16)
        
    import serial
    with serial.Serial(port=serial_port, baudrate=baud_rate, timeout=timeout) as arduino:
        arduino.read_until(startup_message.encode())
        for i in range(number_readings):
            data = arduino.read_until(terminator.encode()) 
            values = data.decode().strip().split(',')
            try:
                numeric_data[i] = data_type(values[0])
                time_diffs[i] = numpy.uint16(values[1]) # NEW Arduino time diff       
            except (ValueError, IndexError): 
                numeric_data[i] = error_value 
                time_diffs[i] = 0 
                
    return numeric_data, numpy.cumsum(time_diffs)*1e-6 # New: reconstruct timestamps from cumulative sum

In [10]:
def read_and_discard(number_to_read=1, number_to_discard=20, **kwds):
    sigs, time = read_arduino_diff_binary(number_readings=number_to_read+number_to_discard, **kwds)
    return sigs[number_to_discard:], time[number_to_discard:]

In [11]:
def read_arduino_diff_binary(number_readings=1, data_type=numpy.float32, error_value=numpy.NaN,
                        serial_port='COM9', baud_rate=115200, timeout=2,
                        terminator='\r\n', startup_message='Yours sincerely, Arduino'): 
   
    if number_readings < 1: number_readings = 1 
    numeric_data = numpy.zeros((number_readings,), dtype=data_type)
    time_diffs = numpy.zeros((number_readings,), dtype=numpy.uint16)
        
    import serial
    with serial.Serial(port=serial_port, baudrate=baud_rate, timeout=timeout) as arduino:
        arduino.read_until(startup_message.encode())
        for i in range(number_readings):
            binary_data = arduino.read(size=4) # NEW: read in fixed-length bytearray
            try:
                numeric_data[i] = data_type((binary_data[0] + (binary_data[1]*2**8))*5/1023) # binary data to voltage
                time_diffs[i] = data_type(binary_data[2] + (binary_data[3]*2**8)) # binary data to time difference
            except (ValueError, IndexError):
                numeric_data[i] = error_value
                time_diffs[i] = 0
    
    timestamp = numpy.cumsum(time_diffs)*1e-6 
    
    return numeric_data, timestamp

In [12]:
def FFT(signal, time, plot=True):
    '''
    Determine and plot (positive frequency) FFT of signal with timebase time (both numpy arrays).
    Returns frequency and amplitude of (non-zero frequency) fundamental tone.
    '''
    try:
        from scipy.fft import fft
    except ModuleNotFoundError:
        from scipy.fftpack import fft  
    import numpy as np
    
    fft_signal = fft(signal) # complex FFT spectrum
    
    yf = 2.0/len(signal) * np.abs(fft_signal[:len(signal)//2])[1:] 
    xf = np.linspace(0.0, len(time)/(2.0*(time[-1]-time[0])), len(time)//2)[1:] 
    phase = np.angle(fft_signal[1:len(signal)//2]) + np.pi/2
    
    peak_freq = xf[np.argmax(yf)]
    amplitude = np.max(yf)
    
    if plot:
        plt.figure(figsize=(6,3))
        plt.plot(xf, yf)
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Amplitude')
        plt.show()

        print('Fundamental frequency at {} Hz, with amplitude {} V.'.format(peak_freq, amplitude))
        print('Nyquist frequency as last point in FFT = {}'.format(xf[-1]))

    return xf, yf, phase

In [13]:
def harmonic_gen(freq=120, length=10.0):
    '''
    freq -> fundamental frequency in Hz, 
    length -> signal duration in seconds
    '''
    import numpy as np
    from simpleaudio import play_buffer
    
    sample_rate = 44100 # Hz - CD quality
    samples = int(length)*sample_rate
    time = np.linspace(0, int(length), samples, False)
    
    audio = np.zeros((samples, 2))
    
    for i in range(5): # NEW: add in some harmonics!
        f = i*freq
        audio[:,0] += np.sin(2*np.pi*f*time) # 'left' channel
        audio[:,1] += np.cos(2*np.pi*f*time) # 'right' channel
    
    audio *= 32767 / np.max(np.abs(audio)) # normalize to 16-bit range
    audio = audio.astype(np.int16) # convert to 16-bit datatype

    # start playback
    play = play_buffer(audio, 2, 2, sample_rate)
    print('Playing {} s audio signal at {} Hz.'.format(int(length), freq))
    # DON'T wait for playback to finish before exiting

In [14]:
def sinusoid(freq, amp, phase, time):
    return amp*numpy.sin(2*numpy.pi*freq*time + phase)

In [2]:
def read_arduino_diff_binary_4ch(number_readings=1, data_type=numpy.float32, error_value=numpy.NaN,
                                 serial_port='COM10', baud_rate=115200, timeout=2,
                                 terminator='\r\n', startup_message='Yours sincerely, Arduino'): 
   
    if number_readings < 1: number_readings = 1 
    numeric_data = numpy.zeros((number_readings, 4), dtype=data_type) # NEW: array shape
    time_diffs = numpy.zeros((4*number_readings,), dtype=numpy.uint16) # NEW: array length
        
    import serial
    with serial.Serial(port=serial_port, baudrate=baud_rate, timeout=timeout) as arduino:
        arduino.read_until(startup_message.encode())
        for i in range(number_readings):
            binary_data = arduino.read(size=16) # NEW: length of bytearray
            for ch in range(4): # NEW: loop over channels and index data appropriately
                try:
                    numeric_data[i, ch] = data_type((binary_data[4*ch] + (binary_data[4*ch + 1]*2**8))*5/1023) 
                    time_diffs[4*i + ch] = data_type(binary_data[4*ch + 2] + (binary_data[4*ch + 3]*2**8)) 
                except (ValueError, IndexError):
                    numeric_data[i, ch] = error_value
                    time_diffs[4*i + ch] = 0

    timestamp = numpy.cumsum(time_diffs)*1e-6 
    
    return numeric_data, timestamp.reshape(number_readings, 4) # NEW: reshape timestamp array


def read_and_discard(number_to_read=1, number_to_discard=20, **kwds):
    sigs, time = read_arduino_diff_binary_4ch(number_readings=number_to_read+number_to_discard, **kwds)
    return sigs[number_to_discard:], time[number_to_discard:] 

In [1]:
def read_arduino_burst(number_readings=250, data_type=numpy.float32, error_value=numpy.NaN,
                       serial_port='COM10', baud_rate=115200, timeout=None, # NEW: no timeout
                       terminator='\r\n', startup_message='Yours sincerely, Arduino'): 
   
    if number_readings < 1: number_readings = 1 
    numeric_data = numpy.zeros((number_readings,), dtype=data_type) 
    time_diffs = numpy.zeros((number_readings,), dtype=numpy.uint16) 
        
    import serial
    with serial.Serial(port=serial_port, baudrate=baud_rate, timeout=timeout) as arduino:
        arduino.read_until(startup_message.encode())
        
        data_length = number_readings * 4            # NEW: 4 bytes per reading
        binary_data = arduino.read(size=data_length) # NEW: outside loop
          
        for i in range(number_readings):
            try:
                numeric_data[i] = data_type((binary_data[4*i] + (binary_data[4*i + 1]*2**8))*5/1023) 
                time_diffs[i] = data_type(binary_data[4*i + 2] + (binary_data[4*i + 3]*2**8)) 
            except (ValueError, IndexError):
                numeric_data[i] = error_value
                time_diffs[i] = 0

    timestamp = numpy.cumsum(time_diffs)*1e-6 
    
    return numeric_data, timestamp

NameError: name 'numpy' is not defined

In [ ]:
def read_star_network(number_readings=1, data_type=numpy.float32, error_value=numpy.NaN,
                      number_channels=2,
                      serial_port='COM9', baud_rate=115200, timeout=2,
                      terminator='\r\n', startup_message='Yours sincerely, Arduino'): 
   
    if number_readings < 1: number_readings = 1 
    if number_channels < 1: number_channels = 1 # NEW: need at least 1 channel
    numeric_data = numpy.zeros((number_readings, number_channels), dtype=data_type) # NEW: added dimension
    time_diffs = numpy.zeros((number_readings,), dtype=numpy.uint16)
        
    import serial
    with serial.Serial(port=serial_port, baudrate=baud_rate, timeout=timeout) as arduino:
        arduino.read_until(startup_message.encode())
        for i in range(number_readings):
            binary_data = arduino.read(size=2*number_channels + 2) # NEW: determine length of bytearray
            
            try:
                for j in range(number_channels): # unpack data from channels
                    numeric_data[i,j] = data_type((binary_data[2*j] + (binary_data[2*j + 1]*2**8))*5/1023) 
                time_diffs[i] = data_type(binary_data[2*j + 2] + (binary_data[2*j + 3]*2**8))
            except (ValueError, IndexError):
                numeric_data[i] = error_value
                time_diffs[i] = error_value
    
    timestamp = numpy.cumsum(time_diffs)*1e-6 
    
    return numeric_data, timestamp


def read_and_discard(number_to_read=1, number_to_discard=20, **kwds):
    sigs, time = read_star_network(number_readings=number_to_read+number_to_discard, **kwds)
    return sigs[number_to_discard:], time[number_to_discard:]

In [ ]:
def plot_signals(signals, times, save = False, title = None):
    '''
    Function to display sampled data with timestamps (seconds).
    '''
    diffs = 1000*numpy.diff(times) # element-wise difference, in ms
    
    plt.figure()
    
#     left = [entry[0] for entry in signals]
#     right = [entry[1] for entry in signals]
#     plt.plot(times, left, label='L')
#     plt.plot(times, right, label='R')
    
    plt.plot(times,signals, label='L')
    
    plt.xlim(0.02,0.04)
    #plt.ylim(1,5.1)
    plt.legend() 
    plt.xlabel('time (seconds)'); plt.ylabel('ADC\nVoltage (V)')

    plt.tight_layout()
    if(save):
        plt.savefig(title, bbox_inches='tight')
    plt.show()